In [ ]:
import os
import shutil
import random
from typing import List, Dict

def create_stratified_and_copy_images_mixed_sources(
    old_image_dir,
    old_annotation_dir,
    new_image_dir,
    new_annotation_dir,
    output_dir,
    annotation_output_dir,
    old_classes,
    new_classes,
    old_data_fraction= 0.5,
    images_per_class = 100,
    random_seed = 42
):
    """
        old_image_dir: Путь к папке, где хранятся изображения старых классов.
        old_annotation_dir: Путь к папке, где хранятся txt файлы аннотаций старых классов.
        new_image_dir: Путь к папке, где хранятся изображения новых классов.
        new_annotation_dir: Путь к папке, где хранятся txt файлы аннотаций новых классов.
        output_dir: Путь к папке, где будет создана новая, сбалансированная структура.
        old_classes: Список номеров старых классов (int).
        new_classes: Список номеров новых классов (int).
        old_data_fraction: Доля данных старых классов в выходном датасете (примерно).
        images_per_class: Максимальное количество изображений, которое будет взято из каждого старого класса.
        random_seed: Для воспроизводимости.
    """

    random.seed(random_seed)

    # создаем выходную директорию (если ее нет)
    os.makedirs(output_dir, exist_ok=True)

    # словарь аннотаций для старых классов
    old_annotations: Dict[str, int] = {}  # filename (без расширения): class_label
    for filename in os.listdir(old_annotation_dir):
        if filename.endswith(".txt"):
            image_name = filename[:-4]  # Убираем ".txt"
            annotation_path = os.path.join(old_annotation_dir, filename)
            try:
                with open(annotation_path, 'r') as f:
                    line = f.readline().strip()
                    class_label = int(line.split()[0])
                    old_annotations[image_name] = class_label
            except Exception as e:
                print(f"Error reading annotation file {filename} in old annotations: {e}")

    # изображения старых классов по классам (используя словарь аннотаций)
    old_class_images: Dict[int, List[str]] = {c: [] for c in old_classes}
    for image_filename in os.listdir(old_image_dir):
        image_name_without_extension = image_filename.split(".")[0]
        if image_name_without_extension in old_annotations:
            class_label = old_annotations[image_name_without_extension]
            old_class_images[class_label].append(image_filename)

    # список изображений новых классов
    new_class_images = [img for img in os.listdir(new_image_dir) if os.path.isfile(os.path.join(new_image_dir, img))]

    # изображения и аннотации НОВЫХ классов
    total_new_images = 0
    for image_filename in new_class_images:
        image_name_without_extension = image_filename.split(".")[0]
            
        src_image_file = os.path.join(new_image_dir, image_filename)
        dest_image_file = os.path.join(output_dir, image_filename)
        if os.path.isfile(src_image_file):
            shutil.copy2(src_image_file, dest_image_file)
            total_new_images += 1

        src_annotation_file = os.path.join(new_annotation_dir, image_name_without_extension + ".txt")
        dest_annotation_file = os.path.join(annotation_output_dir, image_name_without_extension + ".txt")
        if os.path.isfile(src_annotation_file):
            shutil.copy2(src_annotation_file, dest_annotation_file)


    # копируем изображения и аннотации СТАРЫХ классов (стратифицированно)
    total_old_images = int(total_new_images * (old_data_fraction / (1 - old_data_fraction)))
    images_per_class = min(images_per_class, total_old_images // len(old_classes) + 1)

    for class_label, image_list in old_class_images.items():
        num_images_to_copy = min(images_per_class, len(image_list))
        selected_images = random.sample(image_list, num_images_to_copy)

        for image_filename in selected_images:
            
            src_image_file = os.path.join(old_image_dir, image_filename)
            dest_image_file = os.path.join(output_dir, image_filename)
            if os.path.isfile(src_image_file):
                shutil.copy2(src_image_file, dest_image_file)

            image_name_without_extension = image_filename.split(".")[0]
            src_annotation_file = os.path.join(old_annotation_dir, image_name_without_extension + ".txt")
            dest_annotation_file = os.path.join(annotation_output_dir, image_name_without_extension + ".txt")
            if os.path.isfile(src_annotation_file):
                shutil.copy2(src_annotation_file, dest_annotation_file)


    print(f"Все изображения и аннотации скопированы в: {output_dir}")


# ниже нужно ввести ваши пути до папок
old_image_dir = "test_images_old" # папка для фото в старом датасете
old_annotation_dir = "old_annotations" # папка для аннотаций в старом датасете
new_image_dir = "new_images" # папка с изображениями в новом датасете
new_annotation_dir = "new_annotations" # папка с аннотациями в новом датасете
output_dir = "stratified_images" # папка для фото итогового датасета
output_annotation_dir = "" # папка для аннотаций в итоговом датасете


old_classes = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
new_classes = [10]


create_stratified_and_copy_images_mixed_sources(
    old_image_dir=old_image_dir,
    old_annotation_dir=old_annotation_dir,
    new_image_dir=new_image_dir,
    new_annotation_dir=new_annotation_dir,
    output_dir=output_dir,
    output_annotation_dir=output_annotation_dir,
    old_classes=old_classes,
    new_classes=new_classes,
    old_data_fraction=0.5,
    images_per_class=100,
    random_seed=42
)

In [ ]:
from collections import defaultdict
from sklearn.model_selection import train_test_split

def stratified_split(image_dir, annotation_dir, output_dir, test_size=0.1):
    image_files = [f for f in os.listdir(image_dir) if os.path.isfile(os.path.join(image_dir, f))]
    image_classes = defaultdict(list)

    for image_file in image_files:
        image_name, image_ext = os.path.splitext(image_file)
        annotation_file = os.path.join(annotation_dir, image_name + ".txt")
        classes_in_image = set()
        
        try:
            with open(annotation_file, 'r') as f:
                for line in f:
                    class_id = int(line.split()[0])
                    classes_in_image.add(str(class_id))
        except Exception as e:
            print(f"{annotation_file}: {e}")
            continue

        for class_name in classes_in_image:
            image_classes[class_name].append(image_file)

    train_dir = os.path.join(output_dir, "train")
    test_dir = os.path.join(output_dir, "test")
    train_image_dir = os.path.join(train_dir, "images")
    train_annotation_dir = os.path.join(train_dir, "labels")
    test_image_dir = os.path.join(test_dir, "images")
    test_annotation_dir = os.path.join(test_dir, "labes")

    os.makedirs(train_image_dir, exist_ok=True)
    os.makedirs(train_annotation_dir, exist_ok=True)
    os.makedirs(test_image_dir, exist_ok=True)
    os.makedirs(test_annotation_dir, exist_ok=True)

    train_files = []
    test_files = []

    for class_name, files in image_classes.items():
        train, test = train_test_split(files, test_size=test_size)
        train_files.extend(train)
        test_files.extend(test)

    def copy_files(files, image_dest_dir, annotation_dest_dir):
        for image_file in files:
            image_name, image_ext = os.path.splitext(image_file)
            annotation_file = os.path.join(annotation_dir, image_name + ".txt")
            shutil.copy(os.path.join(image_dir, image_file), os.path.join(image_dest_dir, image_file))
            shutil.copy(annotation_file, os.path.join(annotation_dest_dir, os.path.basename(annotation_file)))
            
    copy_files(train_files, train_image_dir, train_annotation_dir)
    copy_files(test_files, test_image_dir, test_annotation_dir)

    print(f"Обучающая выборка: {len(train_files)} изображений")
    print(f"Тестовая выборка: {len(test_files)} изображений")

image_dir = output_dir
annotation_dir = output_annotation_dir
output_dir1 = ""
stratified_split(image_dir, annotation_dir, output_dir, test_size=0.2)